In [1]:
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.io import ascii
from astropy.table import setdiff, Table, unique, vstack
import numpy as np
%run Crossmatch.ipynb
%run Utility.ipynb

In [24]:
# This is the most advanced code. It takes into account triplicates and the sources which have already contributed to an averaging
# It also takes into account not to perform cross-match between two chips twice (one with one chip being the main, and the second time with it being the secondary catalog)

# The final table of all the duplicates across the chips will be stored in this variable

total_duplicates = Table()

totally_duplicates = Table()

full_cat = ascii.read('/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP_CT.dat')

averaged = Table.copy(full_cat)

# The path where the dataset is located

path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term/'

# Encodes the path of the dataset to be used by other OS methods

folder = os.fsencode(path)

# The iterator for the entries of the catalog of duplicates after the SEARCH_AROUND_SKY method

i = 0

# The iterator for the number of sources found around a same duplicate (i.e., the number of sources being averaged)

j = 1

# The iterator for the entry of the catalog of averaged duplicate sources to be saved in

k = 0

# The RA variable to be averaged across the duplicates

ra = []

# The DEC variable to be averaged across the duplicates

dec = []

# The I-band magnitude variable to be averaged across the duplicates

i_mag = []

# The I-band magnitude error variable to be averaged across the duplicates

i_mag_err = []

# The I-band flux variable to be averaged across the duplicates

i_flux = []

# The I-band flux error variable to be averaged across the duplicates

i_flux_err = []

# The I-band flux radius variable to be averaged across the duplicates

i_flux_radius = []

# The elongation variable to be averaged across the duplicates

elongation = []

# The ellipticity variable to be averaged across the duplicates

ellipticity = []

# The signal-to-noise ratio win variable to be averaged across the duplicates

snr_win = []

# The I-band flux-aperture variable to be averaged across the duplicates

i_flux_aper = []

# The I-band flux-aperture error variable to be averaged across the duplicates

i_flux_aper_err = []

# The RA coordinates which already have been averaged

ra_done = []

# The DEC coordinates which already have been averaged

dec_done = []

# The RA coordinates which already have been taken into account for the future average during the current duplicate source iteration

ra_done_temp = []

# The DEC coordinates which already have been taken into account for the future average during the current duplicate source iteration

dec_done_temp = []

# A list to store which detectors have already contributed to an average over a given duplicate iteration

chip_done_temp = []

# A variable to check if the cycle is performing an average or if it has already performed an average and is waiting for a new cycle to start

check = 0

pairs_done = []

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in os.listdir(folder):
    
    item = os.fsdecode(item) # Decodes the current item so it becomes a string
    
    item = path + item # To comply with the func() function parameter
    
    # The name of the current catalog
    
    item_name = os.path.splitext(os.path.basename(item))[0]
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The current catalog will now be cross-matched with all the other catalogs in the folder
    
    for item_scnd in os.listdir(folder):
        
        item_scnd = os.fsdecode(item_scnd) # Decodes the current second item so it becomes a string
        
        item_scnd = path + item_scnd # To comply with the func() function parameter
        
        # The name of the current second catalog
        
        item_scnd_name = os.path.splitext(os.path.basename(item_scnd))[0]
        
        # If the second catalog is the same as the current catalog, this will iterate the cycle onto the next catalog
        
        if (item_scnd_name == item_name):
            
            pass
        
        # If the second catalog is not the current catalog
        
        else:
            
            # Opens the current second catalog as an astropy.Table object
            
            cat_scnd = ascii.read(item_scnd)
            
            # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
            
            temp_duplicates = cross_match(cat, 'ra', 'dec', cat_scnd, 'ra', 'dec', 3, sep = True)
            
            # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
            
            if len(temp_duplicates) != 0:
                
                # Resets the columns of the current catalog back to their original names
                
                temp_duplicates = reset_main_catalog_columns(cat, temp_duplicates)
                
                # Removes the columns from the cross-matched second catalog
                
                #remove_new_columns(cat, temp_duplicates)
                
                # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
                
                totally_duplicates = vstack([totally_duplicates, temp_duplicates])
                
                if (item_scnd_name + item_name) in pairs_done:
                    
                    pairs_done.append(item_name + item_scnd_name)
                    
                    pass
                
                else:
                    
                    total_duplicates = vstack([total_duplicates, temp_duplicates])
                    
                    pairs_done.append(item_name + item_scnd_name)

total_duplicates = total_duplicates.group_by(['ra_2', 'dec_2'])

i = 0

while i < len(total_duplicates):
    
    initial_i = i
    
    min_d2d = total_duplicates['d2d'][i]
    
    if i + 1 != len(total_duplicates):
        
        while ((total_duplicates['ra_2'][i + 1] == total_duplicates['ra_2'][i]) and (total_duplicates['dec_2'][i + 1] == total_duplicates['dec_2'][i])):
            
            if total_duplicates['d2d'][i + 1] < total_duplicates['d2d'][i]:
                
                min_d2d = total_duplicates['d2d'][i + 1]
                
                min_i = i + 1
                
                i = i + 1
            
            else:
                
                i = i + 1
    
    final_i = i
    
    if initial_i != final_i:
        
        for n in range(initial_i, final_i):
            
            if n == min_i:
                
                pass
            
            else:
                
                total_duplicates.remove_rows(n)
    
    i = i + 1

i = 0

while i < len(total_duplicates):
    
    if total_duplicates['ra'][i] in total_duplicates['ra_2'] and total_duplicates['dec'][i] in total_duplicates['dec_2'] and total_duplicates['ra_2'][i] in total_duplicates['ra'] and total_duplicates['dec_2'][i] in total_duplicates['dec']:
        
        total_duplicates.remove_rows(i)
    
    i = i + 1

total_duplicates = total_duplicates.group_by(['ra', 'dec'])
#print(averaged_temp)
check = -1

i = 0

j = 1

while i < len(total_duplicates):
    #print(len(averaged), len(total_duplicates), len(averaged_temp), i, k)
    
    # If it is the last entry and no average is currently being done
    
    if i + 1 == len(total_duplicates) and check == 2:
        
        i = i + 1
    
    # If it is the last entry and an average currently needs to be done
    
    elif (i + 1 == len(total_duplicates) and (check == 0)):
        # If the current match has not been used for an average, yet
        if (total_duplicates['ra_2'][i] not in ra_done and total_duplicates['dec_2'][i] not in dec_done):
            
            # The cumulative RA variable is added with the RA value of the current source
            
            ra.append(total_duplicates['ra_2'][i])
            
            # The cumulative DEC variable is added with the RA value of the current source
            
            dec.append(total_duplicates['dec_2'][i])
            
            # The cumulative I-band magnitude variable is added with the RA value of the current source
            
            i_mag.append(total_duplicates['I_MAG_SUBARU_2'][i])
            
            i_mag_err.append(total_duplicates['I_MAG_SUBARU_ERR_2'][i] ** 2)
            
            i_flux.append(total_duplicates['I_FLUX_SUBARU_2'][i])
            
            i_flux_err.append(total_duplicates['I_FLUX_SUBARU_ERR_2'][i] ** 2)
            
            i_flux_radius.append(total_duplicates['I_FLUX_RADIUS_SUBARU_2'][i])
            
            elongation.append(total_duplicates['ELONGATION_2'][i])
            
            ellipticity.append(total_duplicates['ELLIPTICITY_2'][i])
            
            snr_win.append(total_duplicates['I_SNR_WIN_SUBARU_2'][i])
            
            i_flux_aper.append(total_duplicates['I_FLUX_APER_SUBARU_2'][i])
            
            i_flux_aper_err.append(total_duplicates['I_FLUX_APER_SUBARU_ERR_2'][i] ** 2)
            
            # The RA of the current source is appended onto the list of RA coordinates of the current block being averaged
            
            ra_done_temp.append(total_duplicates['ra_2'][i])
            
            # The DEC of the current source is appended onto the list of DEC coordinates of the current block being averaged
            
            dec_done_temp.append(total_duplicates['dec_2'][i])
            
            j = j + 1
        
        # Performs the average of the RA of all sources around the current duplicate
        
        averaged['ra'][k] = sum(ra) / j
        
        # Performs the average of the DEC of all sources around the current duplicate
        
        averaged['dec'][k] = sum(dec) / j
        
        # Performs the average of the I-band magnitude of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU'][k] = sum(i_mag) / j
        
        # Performs the average of the I-band magnitude error of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_mag_err))
        
        # Performs the average of the I-band flux of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU'][k] = sum(i_flux) / j
        
        # Performs the average of the I-band flux error of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_err))
        
        # Performs the average of the I-band flux radius of all sources around the current duplicate
        
        averaged['I_FLUX_RADIUS_SUBARU'][k] = sum(i_flux_radius) / j
        
        # Performs the average of the elongation of all sources around the current duplicate
        
        averaged['ELONGATION'][k] = sum(elongation) / j
        
        # Performs the average of the ellipticity of all sources around the current duplicate
        
        averaged['ELLIPTICITY'][k] = sum(ellipticity) / j
        
        # Performs the average of the signal-to-noise ratio of all sources around the current duplicate
        
        averaged['I_SNR_WIN_SUBARU'][k] = sum(snr_win) / j
        
        # Performs the average of the I-band flux-aperture of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU'][k] = sum(i_flux_aper) / j
        
        # Performs the average of the I-band flux-aperture error of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_aper_err))
        
        # Resets the variables for the next duplicate iteration
        
        ra = []
        
        dec = []
        
        i_mag = []
        
        i_mag_err = []
        
        i_flux = []
        
        i_flux_err = []
        
        i_flux_radius = []
        
        elongation = []
        
        ellipticity = []
        
        snr_win = []
        
        i_flux_aper = []
        
        i_flux_aper_err = []
        
        # Cycles through to the next duplicate
        
        i = i + 1
        
        # Resets the number of duplicate sources
        
        j = 1
        
        # Cycles through the next entry on the catalog of averaged duplicates
        
        k = k + 1
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        ra_done.extend(ra_done_temp)
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        dec_done.extend(dec_done_temp)
        
        # Resets the temporary list where the RA values of the current block are stored
        
        ra_done_temp = []
        
        # Resets the temporary list where the DEC values of the current block are stored
        
        dec_done_temp = []
        
        check = 2
    
    # If the current source has already been used for an average and no average is currently being done
    
    elif (total_duplicates['ra'][i] in ra_done and total_duplicates['dec'][i] in dec_done) and (check == 2):
        print("In_1", i, k)
        i = i + 1
    
    # If the next source remains the same and an average currently needs to be done, but the current match has already been used for an average
    
    elif (total_duplicates['ra'][i + 1] == total_duplicates['ra'][i] and total_duplicates['dec'][i + 1] == total_duplicates['dec'][i]) and (total_duplicates['ra_2'][i] in ra_done and total_duplicates['dec_2'][i] in dec_done) and (check == 0):
        
        i = i + 1
    
    # If the next source changes and an average currently needs to be done, but the current match has already been used for an average
    
    elif ((total_duplicates['ra'][i + 1] != total_duplicates['ra'][i] and total_duplicates['dec'][i + 1] != total_duplicates['dec'][i]) and (total_duplicates['ra_2'][i] in ra_done and total_duplicates['dec_2'][i] in dec_done)) and (check == 0):
        
        # Performs the average of the RA of all sources around the current duplicate
        
        averaged['ra'][k] = sum(ra) / j
        
        # Performs the average of the DEC of all sources around the current duplicate
        
        averaged['dec'][k] = sum(dec) / j
        
        # Performs the average of the I-band magnitude of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU'][k] = sum(i_mag) / j
        
        # Performs the average of the I-band magnitude error of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_mag_err))
        
        # Performs the average of the I-band flux of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU'][k] = sum(i_flux) / j
        
        # Performs the average of the I-band flux error of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_err))
        
        # Performs the average of the I-band flux radius of all sources around the current duplicate
        
        averaged['I_FLUX_RADIUS_SUBARU'][k] = sum(i_flux_radius) / j
        
        # Performs the average of the elongation of all sources around the current duplicate
        
        averaged['ELONGATION'][k] = sum(elongation) / j
        
        # Performs the average of the ellipticity of all sources around the current duplicate
        
        averaged['ELLIPTICITY'][k] = sum(ellipticity) / j
        
        # Performs the average of the signal-to-noise ratio of all sources around the current duplicate
        
        averaged['I_SNR_WIN_SUBARU'][k] = sum(snr_win) / j
        
        # Performs the average of the I-band flux-aperture of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU'][k] = sum(i_flux_aper) / j
        
        # Performs the average of the I-band flux-aperture error of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_aper_err))
        
        # Resets the variables for the next duplicate iteration
        
        ra = []
        
        dec = []
        
        i_mag = []
        
        i_mag_err = []
        
        i_flux = []
        
        i_flux_err = []
        
        i_flux_radius = []
        
        elongation = []
        
        ellipticity = []
        
        snr_win = []
        
        i_flux_aper = []
        
        i_flux_aper_err = []
        
        # Cycles through to the next duplicate
        
        i = i + 1
        
        # Resets the number of duplicate sources
        
        j = 1
        
        # Cycles through the next entry on the catalog of averaged duplicates
        
        k = k + 1
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        ra_done.extend(ra_done_temp)
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        dec_done.extend(dec_done_temp)
        
        # Resets the temporary list where the RA values of the current block are stored
        
        ra_done_temp = []
        
        # Resets the temporary list where the DEC values of the current block are stored
        
        dec_done_temp = []
        
        check = 2
    
    # If the next source changes and an average currently needs to be done and the current match has not yet been used for an average
    
    elif (total_duplicates['ra'][i + 1] != total_duplicates['ra'][i] and total_duplicates['dec'][i + 1] != total_duplicates['dec'][i]) and (check == 0):
        print("In_2", i, k)
        
        # The cumulative RA variable is added with the RA value of the current source
        
        ra.append(total_duplicates['ra_2'][i])
        
        # The cumulative DEC variable is added with the RA value of the current source
        
        dec.append(total_duplicates['dec_2'][i])
        
        # The cumulative I-band magnitude variable is added with the RA value of the current source
        
        i_mag.append(total_duplicates['I_MAG_SUBARU_2'][i])
        
        i_mag_err.append(total_duplicates['I_MAG_SUBARU_ERR_2'][i] ** 2)
        
        i_flux.append(total_duplicates['I_FLUX_SUBARU_2'][i])
        
        i_flux_err.append(total_duplicates['I_FLUX_SUBARU_ERR_2'][i] ** 2)
        
        i_flux_radius.append(total_duplicates['I_FLUX_RADIUS_SUBARU_2'][i])
        
        elongation.append(total_duplicates['ELONGATION_2'][i])
        
        ellipticity.append(total_duplicates['ELLIPTICITY_2'][i])
        
        snr_win.append(total_duplicates['I_SNR_WIN_SUBARU_2'][i])
        
        i_flux_aper.append(total_duplicates['I_FLUX_APER_SUBARU_2'][i])
        
        i_flux_aper_err.append(total_duplicates['I_FLUX_APER_SUBARU_ERR_2'][i] ** 2)
        
        # The RA of the current source is appended onto the list of RA coordinates of the current block being averaged
        
        ra_done_temp.append(total_duplicates['ra_2'][i])
        
        # The DEC of the current source is appended onto the list of DEC coordinates of the current block being averaged
        
        dec_done_temp.append(total_duplicates['dec_2'][i])
        
        j = j + 1
        print("In!", j)
        # Performs the average of the RA of all sources around the current duplicate
        
        averaged['ra'][k] = sum(ra) / j
        
        # Performs the average of the DEC of all sources around the current duplicate
        
        averaged['dec'][k] = sum(dec) / j
        
        # Performs the average of the I-band magnitude of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU'][k] = sum(i_mag) / j
        
        # Performs the average of the I-band magnitude error of all sources around the current duplicate
        
        averaged['I_MAG_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_mag_err))
        
        # Performs the average of the I-band flux of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU'][k] = sum(i_flux) / j
        
        # Performs the average of the I-band flux error of all sources around the current duplicate
        
        averaged['I_FLUX_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_err))
        
        # Performs the average of the I-band flux radius of all sources around the current duplicate
        
        averaged['I_FLUX_RADIUS_SUBARU'][k] = sum(i_flux_radius) / j
        
        # Performs the average of the elongation of all sources around the current duplicate
        
        averaged['ELONGATION'][k] = sum(elongation) / j
        
        # Performs the average of the ellipticity of all sources around the current duplicate
        
        averaged['ELLIPTICITY'][k] = sum(ellipticity) / j
        
        # Performs the average of the signal-to-noise ratio of all sources around the current duplicate
        
        averaged['I_SNR_WIN_SUBARU'][k] = sum(snr_win) / j
        
        # Performs the average of the I-band flux-aperture of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU'][k] = sum(i_flux_aper) / j
        
        # Performs the average of the I-band flux-aperture error of all sources around the current duplicate
        
        averaged['I_FLUX_APER_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_aper_err))
        
        # Resets the variables for the next duplicate iteration
        
        ra = []
        
        dec = []
        
        i_mag = []
        
        i_mag_err = []
        
        i_flux = []
        
        i_flux_err = []
        
        i_flux_radius = []
        
        elongation = []
        
        ellipticity = []
        
        snr_win = []
        
        i_flux_aper = []
        
        i_flux_aper_err = []
        
        # Cycles through to the next duplicate
        
        i = i + 1
        
        # Resets the number of duplicate sources
        
        j = 1
        
        # Cycles through the next entry on the catalog of averaged duplicates
        
        k = k + 1
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        ra_done.extend(ra_done_temp)
        
        # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
        
        dec_done.extend(dec_done_temp)
        
        # Resets the temporary list where the RA values of the current block are stored
        
        ra_done_temp = []
        
        # Resets the temporary list where the DEC values of the current block are stored
        
        dec_done_temp = []
        
        check = 2
    
    # When initiating a new average
    
    else:
        print("In_3", i, k)
        if check == 1 or check == -1 or check == 2:
            
            check = 0
            
            ra.append(total_duplicates['ra'][i])
            
            dec.append(total_duplicates['dec'][i])
            
            i_mag.append(total_duplicates['I_MAG_SUBARU'][i])
            
            i_mag_err.append(total_duplicates['I_MAG_SUBARU_ERR'][i] ** 2)
            
            i_flux.append(total_duplicates['I_FLUX_SUBARU'][i])
            
            i_flux_err.append(total_duplicates['I_FLUX_SUBARU_ERR'][i] ** 2)
            
            i_flux_radius.append(total_duplicates['I_FLUX_RADIUS_SUBARU'][i])
            
            elongation.append(total_duplicates['ELONGATION'][i])
            
            ellipticity.append(total_duplicates['ELLIPTICITY'][i])
            
            snr_win.append(total_duplicates['I_SNR_WIN_SUBARU'][i])
            
            i_flux_aper.append(total_duplicates['I_FLUX_APER_SUBARU'][i])
            
            i_flux_aper_err.append(total_duplicates['I_FLUX_APER_SUBARU_ERR'][i] ** 2)
            
            ra_done_temp.append(total_duplicates['ra'][i])
            
            dec_done_temp.append(total_duplicates['dec'][i])
        
        # The cumulative RA variable is added with the RA value of the current source
        
        ra.append(total_duplicates['ra_2'][i])
        
        # The cumulative DEC variable is added with the RA value of the current source
        
        dec.append(total_duplicates['dec_2'][i])
        
        # The cumulative I-band magnitude variable is added with the RA value of the current source
        
        i_mag.append(total_duplicates['I_MAG_SUBARU_2'][i])
        
        i_mag_err.append(total_duplicates['I_MAG_SUBARU_ERR_2'][i] ** 2)
        
        i_flux.append(total_duplicates['I_FLUX_SUBARU_2'][i])
        
        i_flux_err.append(total_duplicates['I_FLUX_SUBARU_ERR_2'][i] ** 2)
        
        i_flux_radius.append(total_duplicates['I_FLUX_RADIUS_SUBARU_2'][i])
        
        elongation.append(total_duplicates['ELONGATION_2'][i])
        
        ellipticity.append(total_duplicates['ELLIPTICITY_2'][i])
        
        snr_win.append(total_duplicates['I_SNR_WIN_SUBARU_2'][i])
        
        i_flux_aper.append(total_duplicates['I_FLUX_APER_SUBARU_2'][i])
        
        i_flux_aper_err.append(total_duplicates['I_FLUX_APER_SUBARU_ERR_2'][i] ** 2)
        
        # The RA of the current source is appended onto the list of RA coordinates of the current block being averaged
        
        ra_done_temp.append(total_duplicates['ra_2'][i])
        
        # The DEC of the current source is appended onto the list of DEC coordinates of the current block being averaged
        
        dec_done_temp.append(total_duplicates['dec_2'][i])
        
        j = j + 1
        
        if (i + 1 == len(total_duplicates)) or (total_duplicates['ra'][i + 1] != total_duplicates['ra'][i] and total_duplicates['dec'][i + 1] != total_duplicates['dec'][i]):
            print("In!", j)
            # Performs the average of the RA of all sources around the current duplicate
            
            averaged['ra'][k] = sum(ra) / j
            
            # Performs the average of the DEC of all sources around the current duplicate
            
            averaged['dec'][k] = sum(dec) / j
            
            # Performs the average of the I-band magnitude of all sources around the current duplicate
            
            averaged['I_MAG_SUBARU'][k] = sum(i_mag) / j
            
            # Performs the average of the I-band magnitude error of all sources around the current duplicate
            
            averaged['I_MAG_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_mag_err))
            
            # Performs the average of the I-band flux of all sources around the current duplicate
            
            averaged['I_FLUX_SUBARU'][k] = sum(i_flux) / j
            
            # Performs the average of the I-band flux error of all sources around the current duplicate
            
            averaged['I_FLUX_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_err))
            
            # Performs the average of the I-band flux radius of all sources around the current duplicate
            
            averaged['I_FLUX_RADIUS_SUBARU'][k] = sum(i_flux_radius) / j
            
            # Performs the average of the elongation of all sources around the current duplicate
            
            averaged['ELONGATION'][k] = sum(elongation) / j
            
            # Performs the average of the ellipticity of all sources around the current duplicate
            
            averaged['ELLIPTICITY'][k] = sum(ellipticity) / j
            
            # Performs the average of the signal-to-noise ratio of all sources around the current duplicate
            
            averaged['I_SNR_WIN_SUBARU'][k] = sum(snr_win) / j
            
            # Performs the average of the I-band flux-aperture of all sources around the current duplicate
            
            averaged['I_FLUX_APER_SUBARU'][k] = sum(i_flux_aper) / j
            
            # Performs the average of the I-band flux-aperture error of all sources around the current duplicate
            
            averaged['I_FLUX_APER_SUBARU_ERR'][k] = np.sqrt((j ** (- 2)) * sum(i_flux_aper_err))
            
            # Resets the variables for the next duplicate iteration
            
            ra = []
            
            dec = []
            
            i_mag = []
            
            i_mag_err = []
            
            i_flux = []
            
            i_flux_err = []
            
            i_flux_radius = []
            
            elongation = []
            
            ellipticity = []
            
            snr_win = []
            
            i_flux_aper = []
            
            i_flux_aper_err = []
            
            # Cycles through to the next duplicate
            
            i = i + 1
            
            # Resets the number of duplicate sources
            
            j = 1
            
            # Cycles through the next entry on the catalog of averaged duplicates
            
            k = k + 1
            
            # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
            
            ra_done.extend(ra_done_temp)
            
            # Adds the RA coordinates of each source in the now-finished block onto the list of done sources
            
            dec_done.extend(dec_done_temp)
            
            # Resets the temporary list where the RA values of the current block are stored
            
            ra_done_temp = []
            
            # Resets the temporary list where the DEC values of the current block are stored
            
            dec_done_temp = []
            
            check = 2
        
        else:
            
            i = i + 1

print("Before:", len(averaged))
while k < len(averaged):
    
    averaged.remove_rows(k)

print("After:", len(averaged))

# A list of columns to be saved in the final astropy.Table of the duplicate sources across our data

columns = ['ra', 'dec', 'ELLIPTICITY', 'ELONGATION', 'I_MAG_SUBARU', 'I_MAG_SUBARU_ERR', 'I_FLUX_SUBARU', 'I_FLUX_SUBARU_ERR', 'I_FLUX_RADIUS_SUBARU', 'I_FLUX_APER_SUBARU', 'I_FLUX_APER_SUBARU_ERR', 'I_SNR_WIN_SUBARU', 'DETECTOR']

# Keep only the columns described in the list above

averaged = make_catalog(averaged, columns)

averaged = unique(averaged, keys = ['ra', 'dec'], keep = 'first', silent = True)

print("After UNIQUE:", len(averaged))
ascii.write(averaged, '/home/yobd/Documents/Temp/Cats/Only_Averaged.dat', format = 'csv', overwrite = True)
singles = setdiff(full_cat, totally_duplicates, keys = ['ra', 'dec'])

averaged = vstack([singles, averaged])

# Saves the new catalog in a file

ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Duplicates_Averaged.dat', format = 'csv', overwrite = True)

In_3 0 0
In! 2
In_3 1 1
In! 2
In_3 2 2
In! 2
In_3 3 3
In! 2
In_3 4 4
In! 2
In_3 5 5
In! 2
In_3 6 6
In! 2
In_3 7 7
In! 2
In_3 8 8
In! 2
In_3 9 9
In! 2
In_3 10 10
In! 2
In_3 11 11
In! 2
In_3 12 12
In! 2
In_3 13 13
In! 2
In_3 14 14
In! 2
In_3 15 15
In! 2
In_3 16 16
In! 2
In_3 17 17
In! 2
In_3 18 18
In! 2
In_3 19 19
In! 2
In_3 20 20
In! 2
In_3 21 21
In! 2
In_3 22 22
In! 2
In_3 23 23
In! 2
In_3 24 24
In! 2
In_3 25 25
In! 2
In_3 26 26
In! 2
In_3 27 27
In! 2
In_3 28 28
In! 2
In_3 29 29
In! 2
In_3 30 30
In! 2
In_3 31 31
In! 2
In_3 32 32
In! 2
In_3 33 33
In! 2
In_3 34 34
In! 2
In_3 35 35
In! 2
In_3 36 36
In! 2
In_3 37 37
In! 2
In_3 38 38
In! 2
In_3 39 39
In! 2
In_3 40 40
In! 2
In_3 41 41
In! 2
In_3 42 42
In! 2
In_3 43 43
In! 2
In_3 44 44
In! 2
In_3 45 45
In! 2
In_3 46 46
In! 2
In_3 47 47
In! 2
In_3 48 48
In! 2
In_3 49 49
In! 2
In_3 50 50
In! 2
In_3 51 51
In! 2
In_3 52 52
In! 2
In_3 53 53
In! 2
In_3 54 54
In! 2
In_3 55 55
In! 2
In_3 56 56
In! 2
In_3 57 57
In! 2
In_3 58 58
In! 2
In_3 59 59
In! 2


In! 2
In_3 510 510
In! 2
In_3 511 511
In! 2
In_3 512 512
In! 2
In_3 513 513
In! 2
In_3 514 514
In! 2
In_3 515 515
In! 2
In_3 516 516
In! 2
In_3 517 517
In! 2
In_3 518 518
In! 2
In_3 519 519
In! 2
In_3 520 520
In! 2
In_3 521 521
In! 2
In_3 522 522
In! 2
In_3 523 523
In! 2
In_3 524 524
In! 2
In_3 525 525
In! 2
In_3 526 526
In! 2
In_3 527 527
In! 2
In_3 528 528
In! 2
In_3 529 529
In! 2
In_3 530 530
In! 2
In_3 531 531
In! 2
In_3 532 532
In! 2
In_3 533 533
In! 2
In_3 534 534
In! 2
In_3 535 535
In! 2
In_3 536 536
In! 2
In_3 537 537
In! 2
In_3 538 538
In! 2
In_3 539 539
In! 2
In_3 540 540
In! 2
In_3 541 541
In! 2
In_3 542 542
In! 2
In_3 543 543
In! 2
In_3 544 544
In! 2
In_3 545 545
In! 2
In_3 546 546
In! 2
In_3 547 547
In! 2
In_3 548 548
In! 2
In_3 549 549
In! 2
In_3 550 550
In! 2
In_3 551 551
In! 2
In_3 552 552
In! 2
In_3 553 553
In! 2
In_3 554 554
In! 2
In_3 555 555
In! 2
In_3 556 556
In! 2
In_3 557 557
In! 2
In_3 558 558
In! 2
In_3 559 559
In! 2
In_3 560 560
In! 2
In_3 561 561
In! 2
In_3 5

In! 2
In_3 1110 1086
In! 2
In_3 1111 1087
In! 2
In_3 1112 1088
In! 2
In_3 1113 1089
In! 2
In_3 1114 1090
In! 2
In_3 1115 1091
In! 2
In_3 1116 1092
In! 2
In_3 1117 1093
In! 2
In_3 1118 1094
In! 2
In_3 1119 1095
In! 2
In_3 1120 1096
In! 2
In_3 1121 1097
In! 2
In_3 1122 1098
In! 2
In_3 1123 1099
In! 2
In_3 1124 1100
In! 2
In_3 1125 1101
In! 2
In_3 1126 1102
In_2 1127 1102
In! 3
In_3 1128 1103
In! 2
In_3 1129 1104
In! 2
In_3 1130 1105
In! 2
In_3 1131 1106
In! 2
In_3 1132 1107
In! 2
In_3 1133 1108
In! 2
In_3 1134 1109
In! 2
In_3 1135 1110
In! 2
In_3 1136 1111
In! 2
In_3 1137 1112
In_3 1139 1113
In! 2
In_1 1140 1114
In_3 1141 1114
In! 2
In_3 1142 1115
In! 2
In_3 1143 1116
In! 2
In_3 1144 1117
In! 2
In_3 1145 1118
In! 2
In_3 1146 1119
In! 2
In_3 1147 1120
In! 2
In_3 1148 1121
In_2 1149 1121
In! 3
In_3 1150 1122
In! 2
In_3 1151 1123
In! 2
In_3 1152 1124
In! 2
In_3 1153 1125
In! 2
In_3 1154 1126
In! 2
In_3 1155 1127
In! 2
In_3 1156 1128
In! 2
In_3 1157 1129
In! 2
In_3 1158 1130
In! 2
In_3 1159 

In! 2
In_3 1590 1556
In! 2
In_3 1591 1557
In! 2
In_3 1592 1558
In! 2
In_3 1593 1559
In! 2
In_3 1594 1560
In! 2
In_3 1595 1561
In! 2
In_3 1596 1562
In! 2
In_3 1597 1563
In! 2
In_3 1598 1564
In! 2
In_3 1599 1565
In! 2
In_3 1600 1566
In! 2
In_3 1601 1567
In! 2
In_3 1602 1568
In! 2
In_3 1603 1569
In! 2
In_3 1604 1570
In! 2
In_3 1605 1571
In! 2
In_3 1606 1572
In! 2
In_3 1607 1573
In! 2
In_3 1608 1574
In! 2
In_3 1609 1575
In! 2
In_3 1610 1576
In! 2
In_3 1611 1577
In! 2
In_3 1612 1578
In! 2
In_3 1613 1579
In! 2
In_3 1614 1580
In! 2
In_3 1615 1581
In! 2
In_3 1616 1582
In! 2
In_3 1617 1583
In! 2
In_3 1618 1584
In! 2
In_3 1619 1585
In! 2
In_3 1620 1586
In! 2
In_3 1621 1587
In! 2
In_3 1622 1588
In! 2
In_3 1623 1589
In! 2
In_3 1624 1590
In! 2
In_3 1625 1591
In! 2
In_3 1626 1592
In! 2
In_3 1627 1593
In! 2
In_3 1628 1594
In! 2
In_3 1629 1595
In! 2
In_3 1630 1596
In! 2
In_3 1631 1597
In! 2
In_3 1632 1598
In! 2
In_3 1633 1599
In! 2
In_3 1634 1600
In! 2
In_3 1635 1601
In! 2
In_3 1636 1602
In! 2
In_3 16

In_3 2057 2021
In! 2
In_3 2058 2022
In! 2
In_3 2059 2023
In! 2
In_3 2060 2024
In! 2
In_3 2061 2025
In! 2
In_3 2062 2026
In! 2
In_3 2063 2027
In! 2
In_3 2064 2028
In! 2
In_3 2065 2029
In! 2
In_3 2066 2030
In! 2
In_3 2067 2031
In! 2
In_3 2068 2032
In! 2
In_3 2069 2033
In! 2
In_3 2070 2034
In! 2
In_3 2071 2035
In! 2
In_3 2072 2036
In! 2
In_3 2073 2037
In! 2
In_3 2074 2038
In! 2
In_3 2075 2039
In! 2
In_3 2076 2040
In! 2
In_3 2077 2041
In! 2
In_3 2078 2042
In! 2
In_3 2079 2043
In! 2
In_3 2080 2044
In! 2
In_3 2081 2045
In! 2
In_3 2082 2046
In! 2
In_3 2083 2047
In! 2
In_3 2084 2048
In! 2
In_3 2085 2049
In! 2
In_3 2086 2050
In! 2
In_3 2087 2051
In! 2
In_3 2088 2052
In! 2
In_3 2089 2053
In! 2
In_3 2090 2054
In! 2
In_3 2091 2055
In! 2
In_3 2092 2056
In! 2
In_3 2093 2057
In! 2
In_3 2094 2058
In! 2
In_3 2095 2059
In! 2
In_3 2096 2060
In! 2
In_3 2097 2061
In! 2
In_3 2098 2062
In! 2
In_3 2099 2063
In! 2
In_3 2100 2064
In! 2
In_3 2101 2065
In! 2
In_3 2102 2066
In! 2
In_3 2103 2067
In! 2
In_3 2104 206

...

In! 2
In_3 2996 2960
In! 2
In_3 2997 2961
In! 2
In_3 2998 2962
In! 2
In_3 2999 2963
In! 2
In_3 3000 2964
In! 2
In_3 3001 2965
In! 2
In_3 3002 2966
In! 2
In_3 3003 2967
In! 2
In_3 3004 2968
In! 2
In_3 3005 2969
In! 2
In_3 3006 2970
In! 2
In_3 3007 2971
In! 2
In_3 3008 2972
In! 2
In_3 3009 2973
In! 2
In_3 3010 2974
In! 2
In_3 3011 2975
In! 2
In_3 3012 2976
In! 2
In_3 3013 2977
In! 2
In_3 3014 2978
In! 2
In_3 3015 2979
In! 2
In_3 3016 2980
In! 2
In_3 3017 2981
In! 2
In_3 3018 2982
In! 2
In_3 3019 2983
In! 2
In_3 3020 2984
In! 2
In_3 3021 2985
In! 2
In_3 3022 2986
In! 2
In_3 3023 2987
In! 2
In_3 3024 2988
In! 2
In_3 3025 2989
In! 2
In_3 3026 2990
In! 2
In_3 3027 2991
In! 2
In_3 3028 2992
In! 2
In_3 3029 2993
In! 2
In_3 3030 2994
In! 2
In_3 3031 2995
In! 2
In_3 3032 2996
In! 2
In_3 3033 2997
In! 2
In_3 3034 2998
In! 2
In_3 3035 2999
In! 2
In_3 3036 3000
In! 2
In_3 3037 3001
In! 2
In_3 3038 3002
In! 2
In_3 3039 3003
In! 2
In_3 3040 3004
In! 2
In_3 3041 3005
In! 2
In_3 3042 3006
In! 2
In_3 30

In! 2
In_3 3412 3375
In! 2
In_3 3413 3376
In! 2
In_3 3414 3377
In! 2
In_3 3415 3378
In! 2
In_3 3416 3379
In! 2
In_3 3417 3380
In! 2
In_3 3418 3381
In! 2
In_3 3419 3382
In! 2
In_3 3420 3383
In! 2
In_3 3421 3384
In! 2
In_3 3422 3385
In! 2
In_3 3423 3386
In! 2
In_3 3424 3387
In! 2
In_3 3425 3388
In! 2
In_3 3426 3389
In! 2
In_3 3427 3390
In! 2
In_3 3428 3391
In! 2
In_3 3429 3392
In! 2
In_3 3430 3393
In! 2
In_3 3431 3394
In! 2
In_3 3432 3395
In! 2
In_3 3433 3396
In! 2
In_3 3434 3397
In! 2
In_3 3435 3398
In! 2
In_3 3436 3399
In! 2
In_3 3437 3400
In! 2
In_3 3438 3401
In! 2
In_3 3439 3402
In! 2
In_3 3440 3403
In! 2
In_3 3441 3404
In! 2
In_3 3442 3405
In! 2
In_3 3443 3406
In! 2
In_3 3444 3407
In! 2
In_3 3445 3408
In! 2
In_3 3446 3409
In! 2
In_3 3447 3410
In! 2
In_3 3448 3411
In! 2
In_3 3449 3412
In! 2
In_3 3450 3413
In! 2
In_3 3451 3414
In! 2
In_3 3452 3415
In! 2
In_3 3453 3416
In! 2
In_3 3454 3417
In! 2
In_3 3455 3418
In! 2
In_3 3456 3419
In! 2
In_3 3457 3420
In! 2
In_3 3458 3421
In! 2
In_3 34

In! 2
In_3 3808 3767
In! 2
In_3 3809 3768
In! 2
In_3 3810 3769
In! 2
In_3 3811 3770
In! 2
In_3 3812 3771
In! 2
In_3 3813 3772
In! 2
In_3 3814 3773
In! 2
In_3 3815 3774
In! 2
In_3 3816 3775
In! 2
In_3 3817 3776
In! 2
In_3 3818 3777
In! 2
In_3 3819 3778
In! 2
In_3 3820 3779
In! 2
In_3 3821 3780
In! 2
In_3 3822 3781
In! 2
In_3 3823 3782
In! 2
In_3 3824 3783
In! 2
In_3 3825 3784
In! 2
In_3 3826 3785
In! 2
In_3 3827 3786
In! 2
In_3 3828 3787
In! 2
In_3 3829 3788
In! 2
In_3 3830 3789
In! 2
In_3 3831 3790
In! 2
In_3 3832 3791
In! 2
In_3 3833 3792
In! 2
In_3 3834 3793
In! 2
In_3 3835 3794
In! 2
In_3 3836 3795
In! 2
In_3 3837 3796
In! 2
In_3 3838 3797
In! 2
In_3 3839 3798
In! 2
In_3 3840 3799
In! 2
In_3 3841 3800
In! 2
In_3 3842 3801
In! 2
In_3 3843 3802
In! 2
In_3 3844 3803
In! 2
In_3 3845 3804
In! 2
In_3 3846 3805
In! 2
In_3 3847 3806
In! 2
In_3 3848 3807
In! 2
In_3 3849 3808
In! 2
In_3 3850 3809
In! 2
In_3 3851 3810
In! 2
In_3 3852 3811
In! 2
In_3 3853 3812
In! 2
In_3 3854 3813
In! 2
In_1 38

In_3 4301 4220
In! 2
In_3 4302 4221
In! 2
In_3 4303 4222
In! 2
In_3 4304 4223
In! 2
In_3 4305 4224
In! 2
In_3 4306 4225
In! 2
In_3 4307 4226
In! 2
In_3 4308 4227
In! 2
In_3 4309 4228
In! 2
In_3 4310 4229
In! 2
In_3 4311 4230
In! 2
In_3 4312 4231
In! 2
In_3 4313 4232
In! 2
In_3 4314 4233
In! 2
In_3 4315 4234
In! 2
In_3 4316 4235
In! 2
In_3 4317 4236
In! 2
In_3 4318 4237
In! 2
In_3 4319 4238
In! 2
In_3 4320 4239
In! 2
In_3 4321 4240
In! 2
In_3 4322 4241
In! 2
In_3 4323 4242
In! 2
In_3 4324 4243
In! 2
In_3 4325 4244
In! 2
In_3 4326 4245
In! 2
In_3 4327 4246
In! 2
In_3 4328 4247
In! 2
In_3 4329 4248
In! 2
In_3 4330 4249
In! 2
In_3 4331 4250
In! 2
In_3 4332 4251
In! 2
In_3 4333 4252
In! 2
In_3 4334 4253
In! 2
In_3 4335 4254
In! 2
In_3 4336 4255
In! 2
In_3 4337 4256
In! 2
In_3 4338 4257
In! 2
In_3 4339 4258
In! 2
In_3 4340 4259
In! 2
In_3 4341 4260
In! 2
In_3 4342 4261
In! 2
In_3 4343 4262
In! 2
In_3 4344 4263
In! 2
In_3 4345 4264
In! 2
In_3 4346 4265
In! 2
In_3 4347 4266
In! 2
In_3 4348 426

In! 2
In_3 4713 4618
In! 2
In_3 4714 4619
In! 2
In_3 4715 4620
In! 2
In_3 4716 4621
In! 2
In_3 4717 4622
In! 2
In_3 4718 4623
In! 2
In_3 4719 4624
In! 2
In_3 4720 4625
In! 2
In_3 4721 4626
In! 2
In_3 4722 4627
In! 2
In_3 4723 4628
In! 2
In_3 4724 4629
In! 2
In_3 4725 4630
In! 2
In_3 4726 4631
In! 2
In_3 4727 4632
In! 2
In_3 4728 4633
In! 2
In_3 4729 4634
In! 2
In_3 4730 4635
In! 2
In_3 4731 4636
In! 2
In_3 4732 4637
In! 2
In_3 4733 4638
In! 2
In_3 4734 4639
In! 2
In_3 4735 4640
In! 2
In_3 4736 4641
In! 2
In_3 4737 4642
In! 2
In_3 4738 4643
In! 2
In_3 4739 4644
In! 2
In_3 4740 4645
In! 2
In_3 4741 4646
In! 2
In_3 4742 4647
In! 2
In_3 4743 4648
In! 2
In_3 4744 4649
In! 2
In_3 4745 4650
In! 2
In_3 4746 4651
In! 2
In_3 4747 4652
In! 2
In_3 4748 4653
In! 2
In_3 4749 4654
In! 2
In_3 4750 4655
In! 2
In_3 4751 4656
In! 2
In_3 4752 4657
In! 2
In_3 4753 4658
In! 2
In_3 4754 4659
In! 2
In_3 4755 4660
In! 2
In_3 4756 4661
In! 2
In_3 4757 4662
In! 2
In_3 4758 4663
In! 2
In_3 4759 4664
In! 2
In_3 47

In [2]:
## NEW CELL WITH SEQUENTIAL AVERAGING ##
%run Crossmatch.ipynb
total_duplicates = Table()

# The path where the dataset is located

#path = '/home/yobd/Documents/Photometry_Output/Zero_Point_Only/'

#path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_New_Run/'

# Path for aperture photometry based PSF-modelled catalogs
path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/'

averaged = ascii.read('/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/Chihiro.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['Clarisse', 'Fio', 'Kiki', 'Nausicaa', 'Ponyo', 'San', 'Satsuki', 'Sheeta', 'Sophie']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    print(item_name)
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU'] = (temp_duplicates['I_MAG_SUBARU'] + temp_duplicates['I_MAG_SUBARU_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_MAG_SUBARU_ERR'] * temp_duplicates['I_MAG_SUBARU_ERR'] + 0.25 * temp_duplicates['I_MAG_SUBARU_ERR_2'] * temp_duplicates['I_MAG_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_SUBARU'] = (temp_duplicates['I_FLUX_SUBARU'] + temp_duplicates['I_FLUX_SUBARU_2']) / 2
        
        temp_duplicates['I_FLUX_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_SUBARU_ERR'] * temp_duplicates['I_FLUX_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_RADIUS_SUBARU'] = (temp_duplicates['I_FLUX_RADIUS_SUBARU'] + temp_duplicates['I_FLUX_RADIUS_SUBARU_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['I_SNR_WIN_SUBARU'] = (temp_duplicates['I_SNR_WIN_SUBARU'] + temp_duplicates['I_SNR_WIN_SUBARU_2']) / 2
        
        #temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        #temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        #temp_duplicates['I_FLUX_APER_SUBARU'] = (temp_duplicates['I_FLUX_APER_SUBARU'] + temp_duplicates['I_FLUX_APER_SUBARU_2']) / 2
        
        #temp_duplicates['I_FLUX_APER_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'])
        
        # Removes the columns from the cross-matched second catalog
        
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

#ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP.dat', format = 'csv', overwrite = True)

#ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP_CT_New_Run.dat', format = 'csv', overwrite = True)


# Path for aperture photometry based PSF-modelled catalogs
ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP_CT_Aper_PSF.dat', format = 'csv', overwrite = True)

print(averaged)

Clarisse
Fio
Kiki
Nausicaa
Ponyo
San
Satsuki
Sheeta
Sophie
        ra                 dec         ...     ksAperMag3Err     DETECTOR
------------------ ------------------- ... --------------------- --------
285.08119619898883  -36.92538333879947 ...    0.1610533744096756 Nausicaa
285.08133561427456  -36.92997296952866 ...   0.13872365653514862 Nausicaa
285.08152146155703 -36.946266021430326 ...   0.17129318416118622 Nausicaa
285.08164700778866   -36.9128840942102 ...    0.1999858170747757 Nausicaa
285.08191778038326  -36.90810164049388 ...          -999999488.0 Nausicaa
285.08208616963077  -36.87260750280702 ...   0.04557235911488533 Nausicaa
 285.0826123766585  -36.84959781376215 ...   0.04723486676812172 Nausicaa
 285.0826325972246  -36.92841693627763 ...          -999999488.0 Nausicaa
 285.0830768103179  -36.83753529766357 ...   0.04441603273153305 Nausicaa
285.08310339170777  -36.87034207038855 ...          -999999488.0 Nausicaa
               ...                 ... ...           

In [3]:
## The duplicates code for the H-Alpha catalog using last cell's method

%run Crossmatch.ipynb
total_duplicates = Table()

# The path where the dataset is located

path = '/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/'

#path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term/'

averaged = ascii.read('/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/N-A-L656_chihiro_stacked_tmp.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['N-A-L656_clarisse_stacked_tmp', 'N-A-L656_fio_stacked_tmp', 'N-A-L656_kiki_stacked_tmp', 'N-A-L656_nausicaa_stacked_tmp', 'N-A-L656_ponyo_stacked_tmp', 'N-A-L656_san_stacked_tmp', 'N-A-L656_satsuki_stacked_tmp', 'N-A-L656_sheeta_stacked_tmp', 'N-A-L656_sophie_stacked_tmp']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    print(item_name)
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['FLUX_PSF'] = (temp_duplicates['FLUX_PSF'] + temp_duplicates['FLUX_PSF_2']) / 2
        
        temp_duplicates['FLUXERR_PSF'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_PSF'] * temp_duplicates['FLUXERR_PSF'] + 0.25 * temp_duplicates['FLUXERR_PSF_2'] * temp_duplicates['FLUXERR_PSF_2'])
        
        temp_duplicates['FLUX_RADIUS'] = (temp_duplicates['FLUX_RADIUS'] + temp_duplicates['FLUX_RADIUS_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['SNR_WIN'] = (temp_duplicates['SNR_WIN'] + temp_duplicates['SNR_WIN_2']) / 2
        
        temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        # Removes the columns from the cross-matched second catalog
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

print("Length of AVERAGED before flag cut:", len(averaged))
averaged.remove_rows([averaged['FLAGS'] != 0])
print("Length of AVERAGED after flag cut:", len(averaged))

ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/N-A-L656_Full_Catalog_Aper_PSF.dat', format = 'csv', overwrite = True)

print(averaged)

N-A-L656_clarisse_stacked_tmp
N-A-L656_fio_stacked_tmp
N-A-L656_kiki_stacked_tmp
N-A-L656_nausicaa_stacked_tmp
N-A-L656_ponyo_stacked_tmp
N-A-L656_san_stacked_tmp
N-A-L656_satsuki_stacked_tmp
N-A-L656_sheeta_stacked_tmp
N-A-L656_sophie_stacked_tmp
Length of AVERAGED before flag cut: 26754
Length of AVERAGED after flag cut: 22969
NUMBER  X_IMAGE   Y_IMAGE  ... SNR_WIN         ra                 dec        
------ --------- --------- ... ------- ------------------ -------------------
    38 2834.3274   404.391 ...   16.46 285.08145135963935 -36.946252566582736
   167 2834.3848 1090.4749 ...   31.41 285.08181148555633   -36.9081215652054
   240 2828.0273 1268.0155 ...    13.3 285.08235218599384  -36.89827641052004
    90 2820.5254  724.6572 ...   29.08 285.08254652093296  -36.92843782185453
   667 2829.4248 1887.1907 ...   25.88  285.0826881832103 -36.864010804457635
   551 2824.3071  1772.493 ...    68.3 285.08294945923797  -36.87034964225969
   587 2824.1995  1811.001 ...   33.67 285.08

/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


In [2]:
## CELL FOR CATALOG BEFORE PHOTOMETRY USING SEQUENTIAL AVERAGING ##

%run Crossmatch.ipynb

total_duplicates = Table()

# The path where the dataset is located

path = '/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+/'

averaged = ascii.read('/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+/W-S-I+_chihiro_stacked_tmp.cat')

averaged.remove_rows([averaged['FLAGS'] != 0])

averaged.remove_rows([averaged['FLUX_AUTO'] == 0])

averaged.remove_rows([averaged['ELLIPTICITY'] >= 0.5])

# Encodes the path of the dataset to be used by other OS methods

folder = ['W-S-I+_clarisse_stacked_tmp', 'W-S-I+_fio_stacked_tmp', 'W-S-I+_kiki_stacked_tmp', 'W-S-I+_nausicaa_stacked_tmp', 'W-S-I+_ponyo_stacked_tmp', 'W-S-I+_san_stacked_tmp', 'W-S-I+_satsuki_stacked_tmp', 'W-S-I+_sheeta_stacked_tmp', 'W-S-I+_sophie_stacked_tmp']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    print(item_name)
    
    item = path + item + '.cat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    cat.remove_rows([cat['FLAGS'] != 0])
    
    cat.remove_rows([cat['FLUX_AUTO'] == 0])
    
    cat.remove_rows([cat['ELLIPTICITY'] >= 0.5])
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['X_IMAGE'] = (temp_duplicates['X_IMAGE'] + temp_duplicates['X_IMAGE_2']) / 2
        
        temp_duplicates['Y_IMAGE'] = (temp_duplicates['Y_IMAGE'] + temp_duplicates['Y_IMAGE_2']) / 2
        
        temp_duplicates['ALPHA_J2000'] = (temp_duplicates['ALPHA_J2000'] + temp_duplicates['ALPHA_J2000_2']) / 2
        
        temp_duplicates['DELTA_J2000'] = (temp_duplicates['DELTA_J2000'] + temp_duplicates['DELTA_J2000_2']) / 2
        
        temp_duplicates['FLUX_AUTO'] = (temp_duplicates['FLUX_AUTO'] + temp_duplicates['FLUX_AUTO_2']) / 2
        
        temp_duplicates['FLUXERR_AUTO'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_AUTO'] * temp_duplicates['FLUXERR_AUTO'] + 0.25 * temp_duplicates['FLUXERR_AUTO_2'] * temp_duplicates['FLUXERR_AUTO_2'])
        
        temp_duplicates['FLUX_RADIUS'] = (temp_duplicates['FLUX_RADIUS'] + temp_duplicates['FLUX_RADIUS_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['SNR_WIN'] = (temp_duplicates['SNR_WIN'] + temp_duplicates['SNR_WIN_2']) / 2
        
        temp_duplicates['FLUX_APER'] = (temp_duplicates['FLUX_APER'] + temp_duplicates['FLUX_APER_2']) / 2
        
        temp_duplicates['FLUXERR_APER'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_APER'] * temp_duplicates['FLUXERR_APER'] + 0.25 * temp_duplicates['FLUXERR_APER_2'] * temp_duplicates['FLUXERR_APER_2'])
        
        # Removes the columns from the cross-matched second catalog
        
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

ascii.write(averaged, '/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+_Full_Catalog_no_duplicates.dat', format = 'csv', overwrite = True)

print(averaged)

/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_clarisse_stacked_tmp
W-S-I+_fio_stacked_tmp
W-S-I+_kiki_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False
/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_nausicaa_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False
/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


...

W-S-I+_satsuki_stacked_tmp
W-S-I+_sheeta_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_sophie_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


NUMBER  X_IMAGE   Y_IMAGE  ...         ra                 dec        
------ --------- --------- ... ------------------ -------------------
   149 2833.5979  529.8074 ...  285.0809916661293 -36.938937629844744
   229 2832.6501  773.8334 ... 285.08120009837006 -36.925378182945174
   306 2833.4841  960.5045 ... 285.08126073184445  -36.91501670009131
   196 2829.9634  691.1179 ...   285.081332207662  -36.92997269382326
   472 2833.3518 1231.3008 ...   285.081450186502  -36.90000286998675
   104 2824.8125  398.0334 ... 285.08151390947955  -36.94626764930463
   311 2827.8784  998.5699 ... 285.08166507228805 -36.912905094181866
   252 2825.5286  809.5094 ... 285.08170463018445 -36.923397534577255
   358 2824.9929 1085.2549 ...  285.0819174153343 -36.908097620533304
   637 2826.5627 1405.7424 ...  285.0820325499118 -36.890341596865966
   ...       ...       ... ...                ...                 ...
  4700   26.1663 2109.6592 ...  285.2739416833508 -37.088459422976285
  3778   27.4028  16